In [1]:
import mpg.mpgml.dataset.generator as mpgml_generator

2023-04-02 04:57:54.603900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from importlib import reload
import mpg.wrapper as mpgwrapper

In [14]:
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfp
import mpg.wrapper as mpgwrapper


def _convert_sparse_matrix_to_sparse_tensor(X, shape_hint):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, shape_hint)


def _stack_sparse_tensors(shape_hint, *A):
    indices = []
    values = []
    for i, Z in enumerate(A):
        for L, v in zip(Z.indices, Z.values):
            indices.append([i, *L])
            values.append(v)
    return tf.sparse.SparseTensor(indices, values, shape_hint)


def _as_tensor(A, as_dense: bool, shape_hint=None):
    if as_dense:
        return tf.convert_to_tensor(A.todense())
    else:
        return _convert_sparse_matrix_to_sparse_tensor(A, shape_hint)


def _generate_instances(n, p, seed, cardinality: int, target: bool, as_graph: bool,
                        adj_matrix: bool, weight_matrix: bool, as_dense: bool):
    generator = np.random.Generator(np.random.MT19937(seed))
    graph = mpg.graph.random_graph.gnp_random_mpg(n=n, p=p, seed=seed, method="fast", loops=True,
                                                  distribution="integers", low=0, high=10, endpoint=True)
    output = None
    if as_graph:
        output = graph
    else:
        if adj_matrix and weight_matrix:
            A = _as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n))
            W = _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n))
            if as_dense:
                output = tf.stack([A, W], axis=0)
            else:
                output = tf.cast(_stack_sparse_tensors((2, n, n), A, W), dtype=tf.float32)

        elif adj_matrix:
            output = tf.cast(_as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n)),
                             dtype=tf.float32)
        elif weight_matrix:
            output = tf.cast(
                _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n)),
                dtype=tf.float32)
    starting = tf.constant([generator.integers(0, n), generator.integers(0, 1, endpoint=True)])
    if target:
        # TODO: Add target
        return (output, starting, 1)
    else:
        return (output, 1)

def cast_all(dtype,*args):
    return tuple(tf.cast(arg, dtype) for arg in args)


def _adj_matrix_generator(n, p):
    A = np.zeros([n, n], dtype=np.uint8)
    for k in range(n):
        A[k, :] = np.random.binomial(1, p, n)
        while A[k, :].sum() == 0:
            A[k, :] = np.random.binomial(1, p, n)
    return A

def _generate_dense_instances(n, p, seeder, cardinality: int, target: bool, weight_matrix: bool, flatten: bool,
                              weight_distribution: tfp.distributions.Distribution, weight_type):
    adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=p, dtype=tf.bool)
    turn_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.5)
    # discrete=tfp.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = weight_distribution.sample(shape, seed=seeder())
    dtype = weight_distribution.dtype
    A = tf.numpy_function(_adj_matrix_generator, inp=[n, p], Tout=tf.uint8,stateful=True)
    if flatten:
        A = tf.reshape(A, shape=(n*n,))
    A = tf.cast(A, dtype=dtype)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,), 0, n, dtype=tf.int32, seed=seeder())
    player = turn_distribution.sample((1,), seed=seeder())
    if flatten:
        if weight_matrix:
            output = tf.concat(cast_all(dtype, A, W, vertex, player), axis=0)
        else:
            output = tf.concat(cast_all(dtype, A, vertex, player), axis=0)
        if target:
            if weight_type == tf.int32 or weight_type == tf.int64:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_int_matrix_flattened_cxx(
                        output.numpy().astype(np.int32).tolist(),"dense"),
                    inp=[output], Tout=tf.int32)
            else:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_float_matrix_flattened_cxx(
                        output.numpy().astype(np.float32).tolist(),"dense"),
                    inp=[output], Tout=tf.float32)
            target_value = tf.reshape(tf.ensure_shape(target_value, ()), shape=(1,))
            return (tf.cast(output, dtype=tf.float32), tf.cast(target_value, dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


class MPGGeneratedDenseDataset(tf.data.Dataset):

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                target: bool = False, weight_matrix: bool = True, flatten=False, seed=None,
                weights_distribution: tfp.distributions.Distribution = None,
                weight_type : str = "int"):
        if weight_type == "int":
            weight_type= tf.int32
        elif weight_type == "float":
            weight_type = tf.float32
        elif weight_type == "double":
            weight_type = tf.float64
        elif not isinstance(weight_type, tf.DType):
            raise ValueError("weight_type must be a string or a tf.DType")
        if weights_distribution is None:
            weights_distribution = tfp.distributions.Uniform(low=-1, high=1)
        if seed is None:
            seed = np.random.randint(0, 1 << 32)

        seeder = tfp.util.SeedStream(seed, "seeding_generator")

        shape = None
        if flatten:
            if weight_matrix:
                shape = (2 * n * n + 2,)
            else:
                shape = (n * n + 2,)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32),)
        else:
            if weight_matrix:
                shape = (2, n, n)
            else:
                shape = (n, n)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))

        generated: tf.data.Dataset
        if cardinality == tf.data.INFINITE_CARDINALITY:
            generated = tf.data.Dataset.counter(start=seed, step=1)
        else:
            generated = tf.data.Dataset.range(seed, seed + cardinality)
        return generated.map(
            lambda seed: _generate_dense_instances(n, p, seeder, cardinality, target, weight_matrix, flatten,
                                                   weights_distribution, weight_type),
            num_parallel_calls=12
        )
        #    range,
        #    args=(n,p,cardinality, target, weight_matrix,flatten),
        #    output_signature=signature
        # )

    def __init__(self, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                 target: bool = False, weight_matrix: bool = True, flatten=False, seed=None,
                 weights_distribution: tfp.distributions.Distribution = None,
                 weight_type : str = "int"):
        self.n = n
        self.p = p
        self.cardinality = cardinality
        self.target = target
        self.weight_matrix = weight_matrix
        self.flatten = flatten
        self.seed = seed
        self.weights_distribution = weights_distribution

    def _permutation(self, x, P):
        if self.flatten:
            S = tf.concat(tf.reshape(tf.tensordot(P, P, axes=None), shape=(-1,)))
            return tf.concat([tf.gather(x, S, axis=0), P[x[-2]], x[-1]])

    def permutation(self, P):
        return self.map(lambda x: self._permutation(x, P))


class MPGGeneratedDataset(tf.data.Dataset):
    def _generator(n, p, cardinality: int, target: bool, as_graph: bool,
                   adj_matrix: bool, weight_matrix: bool, as_dense: bool):
        if cardinality == tf.data.INFINITE_CARDINALITY:
            seed = 0
            while True:
                yield _generate_instances(n, p, seed, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)
                seed += 1
        else:
            for sample_idx in range(cardinality):
                yield _generate_instances(n, p, sample_idx, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY, target: bool = False, as_graph: bool = False,
                adj_matrix: bool = True, weight_matrix: bool = True, as_dense: bool = True):
        shape = None
        if as_graph:
            signature = tf.TensorSpec(shape=(), dtype=mpg.MeanPayoffGraph)
        else:
            if adj_matrix and weight_matrix:
                shape = (2, n, n)
            elif adj_matrix or weight_matrix:
                shape = (n, n)
            else:
                raise ValueError("Must specify at least one of adj_matrix or weight_matrix")
        if as_dense:
            TensorSpec = tf.TensorSpec
        else:
            TensorSpec = tf.SparseTensorSpec
        signature = (TensorSpec(shape=shape), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(n, p, cardinality, target, as_graph, adj_matrix, weight_matrix, as_dense),
            output_signature=signature
        )

    def __init__(self, n, p):
        pass


In [28]:
dataset=MPGGeneratedDenseDataset(10,0.5,target=True,weight_matrix=True,flatten=True,weights_distribution=tfp.distributions.Uniform(-100,100),weight_type="int")
transformed=dataset.batch(64).take(1024).cache().repeat()
validation_dataset=MPGGeneratedDenseDataset(10,0.5,target=True,weight_matrix=True,flatten=True,weights_distribution=tfp.distributions.Uniform(-10,10),weight_type="float").batch(64).take(12).cache()

In [29]:
for x,y in dataset.take(1):
    print(y)

tf.Tensor([1.], shape=(1,), dtype=float32)


In [30]:
import mpg.mpgml.layers.normalisation as mpgml_norm
import mpg.mpgml.layers.augmentation as mpgml_augm
import tensorflow.keras as keras
model=keras.Sequential([
    mpgml_norm.EdgeNormalisationLayer(edges_interval=(1,-2)),
    mpgml_augm.EdgeWeightsNoiseLayer(noise_layer=keras.layers.GaussianNoise(stddev=0.01),edges_interval=(1,-2)),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.Dense(1,"sigmoid")
])

In [31]:
model.compile(
    optimizer="Adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=64,
    shuffle=False,
          validation_data=validation_dataset
         )

Epoch 1/25
64/64 [==============================] - 12s 162ms/step - loss: 0.6858 - accuracy: 0.5676 - val_loss: 0.6641 - val_accuracy: 0.6328
Epoch 2/25
64/64 [==============================] - 8s 130ms/step - loss: 0.6220 - accuracy: 0.6567 - val_loss: 0.6414 - val_accuracy: 0.6315
Epoch 3/25
12/64 [====>.........................] - ETA: 6s - loss: 0.5916 - accuracy: 0.6992

KeyboardInterrupt: 

In [ ]:
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=256,
    shuffle=False,
          validation_data=validation_dataset
         )

In [130]:
x,y=next(iter(transformed))
y

2023-04-04 05:43:34.998569: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.]], dtype=float32)>

In [111]:
graph = mpg.graph.random_graph.gnp_random_mpg(10, 0.5, seed=1, method="fast", loops=True,
                                              distribution="integers", low=0, high=10, endpoint=True)

In [123]:
A = tf.convert_to_tensor(nx.adjacency_matrix(graph, weight=None).todense())

In [2]:
from numba import jit
import networkx as nx
import numpy as np
from typing import Union, TypeVar, Type
import mpg.games.mpg as mpg


1.19 s ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
import mpg.graph.random_graph as rg
G= rg.gnp_random_mpg(500,0.5,seed=27,distribution="integers",low=-10,high=10,endpoint=True)

In [27]:
%timeit mpg.optimal_strategy_pair(G)

20.2 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
import mpg.wrapper as wrapper
%timeit wrapper.optimal_strategy_pair(G)

664 ms ± 63.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:

def _get_winner(A,W,vertex,player):
    edges = []
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i,j] == 1:
                edges.append((i,j,W[i,j]))
    C=mpgwrapper.mpgcpp.winners_double_edges_cxx(edges)
    return C[player][vertex]

In [77]:
adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.57)
d=tfp.distributions.Uniform(-10,10)

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1], dtype=int32)>

In [33]:
bytes(tf.constant([5]))

b'\x05\x00\x00\x00'

In [16]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import time
import mpg.graph.random_graph as rg
times=[]
for k in range(10):
    G=rg.gnp_random_mpg(n=1000,p=0.001,distribution="normal",loc=0,scale=1,loops=True)
    print(f"Processing Graph {k}:",flush=True)
    start=time.time()
    mpgwrapper.mpgcpp.winners_float_edges_cxx([(int(u),int(v),int(G.edges[u,v]["weight"])) for u,v in G.edges])
    end=time.time()
    print(f"Processing Time {end-start}:",flush=True)
    times.append(end-start)

Processing Graph 0:
Processing Time 0.2986409664154053:
Processing Graph 1:
Processing Time 0.2029719352722168:
Processing Graph 2:
Processing Time 0.2950937747955322:
Processing Graph 3:
Processing Time 0.012552738189697266:
Processing Graph 4:
Processing Time 0.24564290046691895:
Processing Graph 5:
Processing Time 0.01729297637939453:
Processing Graph 6:
Processing Time 0.0193173885345459:
Processing Graph 7:
Processing Time 0.3035721778869629:
Processing Graph 8:
Processing Time 0.07518315315246582:
Processing Graph 9:
Processing Time 0.2042698860168457:


In [1]:
from importlib import reload
import mpg.wrapper as mpgwrapper
import time
import mpg.graph.random_graph as rg
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfp
import mpg.wrapper as mpgwrapper


def _adj_matrix_generator(n,p):
    
    A=np.zeros([n,n],dtype=np.uint8)
    for k in range(n):
        A[k,:]=np.random.binomial(1,p,n)
        while A[k,:].sum() == 0:
            A[k,:]=np.random.binomial(1,p,n)
    return A

def _generate_dense_instances(n, p, seeder, cardinality: int, target: bool, weight_matrix: bool, flatten: bool,
                              weight_distribution: tfp.distributions.Distribution, weight_type):
    adjacency_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=p,dtype=tf.bool)
    turn_distribution: tfp.distributions.Distribution = tfp.distributions.Bernoulli(probs=0.5)
    # discrete=tfp.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = weight_distribution.sample(shape, seed=seeder())
    dtype=weight_distribution.dtype
    A=tf.numpy_function(_adj_matrix_generator,inp=[n,p],Tout=tf.uint8,stateful=False)
    if flatten:
        A=tf.reshape(A,shape=(-1,))
    A = tf.cast(A,dtype=dtype)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,), 0, n, dtype=tf.int32, seed=seeder())
    player = turn_distribution.sample((1,), seed=seeder())
    if flatten:
        if weight_matrix:
            output = tf.concat(cast_all(dtype,A, W, vertex, player), axis=0)
        else:
            output = tf.concat(cast_all(dtype,A, vertex, player), axis=0)
        if target:
            if weight_type == tf.int32 or weight_type == tf.int64:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_int_matrix_flattened_cxx(output.numpy().astype(np.int32).tolist()),
                    inp=[output], Tout=tf.int32)
            else:
                target_value = tf.py_function(
                    lambda output: mpgwrapper.mpgcpp.winners_tensorflow_float_matrix_flattened_cxx(output.numpy().astype(np.float32).tolist()),
                    inp=[output], Tout=tf.float32)
            target_value = tf.reshape(tf.ensure_shape(target_value, ()), shape=(1,))
            return (tf.cast(output,dtype=tf.float32), tf.cast(target_value, dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


def cast_all(dtype,*args):
    return tuple(tf.cast(arg, dtype) for arg in args)
times=[]
with tf.device("/device:gpu:0") as device:
    for k in range(10):
        print(f"Processing Graph {k}:",flush=True)
        seeder=tfp.util.SeedStream(k,"k")
        start=time.time()
        _generate_dense_instances(n=200,p=1,seeder=seeder,cardinality=1,target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")
        end=time.time()
        print(f"Processing Time {end-start}:",flush=True)
        times.append(end-start)

2023-04-04 18:41:25.354470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Processing Graph 0:
Processing Time 0.09424042701721191:
Processing Graph 1:
Processing Time 0.01358485221862793:
Processing Graph 2:
Processing Time 0.012871503829956055:
Processing Graph 3:
Processing Time 0.01280069351196289:
Processing Graph 4:
Processing Time 0.012240409851074219:
Processing Graph 5:
Processing Time 0.013983726501464844:
Processing Graph 6:
Processing Time 0.012516021728515625:
Processing Graph 7:
Processing Time 0.013568878173828125:
Processing Graph 8:
Processing Time 0.014420747756958008:
Processing Graph 9:
Processing Time 0.016191720962524414:


In [2]:

def _adj_matrix_generator(n,p):
    A=np.zeros([n,n],dtype=np.uint8)
    for k in range(n):
        A[k,:]=np.random.binomial(1,p,n)
        while A[k,:].sum() == 0:
            A[k,:]=np.random.binomial(1,p,n)
    return A
_adj_matrix_generator(100,0.5)

array([[0, 1, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 0, 1],
       ...,
       [1, 1, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 0, 1, 1],
       [0, 1, 0, ..., 1, 0, 0]], dtype=uint8)

In [3]:
import time
n=100
seeder=tfp.util.SeedStream(4,"g")
X=_generate_dense_instances(n=n,p=0.1,seeder=seeder,cardinality=1,
                          target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-100,100),weight_type="int")
L=[]
Z=X.numpy()
for i in range(n):
    for j in range(n):
        if Z[i+n*j] == 1:
            L.append((i,j,int(Z[i+n*j+n*n])))
a=time.time()
mpgwrapper.mpgcpp.winners_tensorflow_int_matrix_flattened_cxx(X.numpy().astype(int).tolist())
b=time.time()
print(b-a)

max_c=99
radius=99500
estimate=109
2.0294992923736572
max_c=99
radius=99500
estimate=109


In [4]:

print(5)

NameError: name 'X' is not defined

In [59]:
n=100
X=_generate_dense_instances(n=n,p=0.01,seeder=seeder,cardinality=1,
                          target=False,weight_matrix=True,flatten=True,weight_distribution=tfp.distributions.Uniform(-30,30),weight_type="int")

NameError: name 'seeder' is not defined

In [50]:
X

<tf.Tensor: shape=(20002,), dtype=float32, numpy=array([ 0.,  0.,  0., ..., -0., 84.,  0.], dtype=float32)>

In [51]:
Z=X.numpy()
Z=Z[n*n:2*n*n].reshape((n,n))
Z=Z.astype(np.int32)
Z
A=X.numpy()[:n*n].reshape((n,n))

In [52]:
import networkx as nx
import mpg.games.mpg as mpg
G=nx.from_numpy_array(A,create_using=mpg.MeanPayoffGraph)
for e in G.edges:
    u,v=e
    G.edges[u,v]["weight"]=Z[u,v]

In [ ]:
mpgwrapper.mean_payoffs(G)